In [4]:
import DEBpython.pet as pet
import DEBpython.models as models
from DEBpython.visualizer import Plotter
import DEBpython.composition as composition

import warnings
from scipy.integrate import simpson
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from datetime import datetime as dt

import ipywidgets as widgets
from IPython.display import display

#%matplotlib nbagg
%matplotlib inline

In [5]:
warnings.filterwarnings("ignore")
#mammal = pet.Ruminant(**pet.animals['bos_taurus_alentejana'])
mammal = pet.Ruminant(**pet.animals['bos_taurus_angus'])
#mammal = pet.Ruminant(**pet.animals['bos_taurus_limousin'])
#mammal = pet.Ruminant(**pet.animals['bos_taurus_charolais'])
print(mammal)
# print(mammal.check_viability())

Parameters at T=311.75 K
Surface-specific maximum assimilation rate: 8074.34 (J/d.cm^2)
Allocation fraction to soma: 0.9633 (-)
Energy conductance: 0.451337 (cm/d)
Volume-specific somatic maintenance rate: 111.115 (J/d.cm^3)
Specific cost for structure: 7960 (J/cm^3)
Maturity maintenance rate constant: 0.0101893 (d^-1)
Maturity at birth: 4335000.0 (J)
Maturity at puberty: 26080000.0 (J)
Reproduction efficiency: 0.95 (-)

Chemical Reactions:
Assimilation: 11.57 X + 4.957 O2 -> E + 6.027 CO2 + 1.815 H2O + 0.6981 CO(NH2)2 + 2.583 CH4 + 1.266 P
Dissipation: E + 1.087 O2 -> 0.925 CO2 + 0.75 H2O + 0.075 CO(NH2)2
Growth: E + 0.01854 O2 ->  0.9829 V + 0.01577 CO2 + 0.01279 H2O + 0.001279 CO(NH2)2



In [6]:
def compute_metrics(breed, p_Am, kap_X, v):
    mammal = pet.Ruminant(**pet.animals[f'bos_taurus_{breed.lower()}'])
    mammal._p_Am = p_Am
    mammal.kap_X = kap_X
    mammal._v = v
    model = models.RUM(mammal)
    sol = model.simulate(food_function=1, t_span=(0, 440), step_size='auto', initial_state='birth')

    for i, t in enumerate(sol.t):
        if t > 330:
            t1 = sol[t]
            t1_i = i
            break
    t2 = sol.time_instant_sols[-1]
    tfi = -simpson(sol.organic_fluxes[0, t1_i:], sol.t[t1_i:]) * sol.time_instant_sols[-1].organism.comp.X.w
    td = (t2.t - t1.t)
    dfi = tfi / td
    adg = (t2.wet_weight - t1.wet_weight) / td
    fcr = dfi / adg
    rgr = (np.log(t2.wet_weight) - np.log(t1.wet_weight)) / td
    # dfi =
    print(f"DFI: {dfi:.5} g\n"
          f"ADG: {adg:.4} g\n"
          f"FCR: {fcr:.4}\n"
          f"RGR: {rgr * 100:.4} %")
    
def default_values(event):
    # print(event)
    if event['name'] == 'label':
        breed = event['new']
        p_Am_widget.value = pet.animals[f'bos_taurus_{breed.lower()}']['p_Am']
        v_widget.value = pet.animals[f'bos_taurus_{breed.lower()}']['v']

In [7]:
cow_selector = widgets.Dropdown(options=['Alentejana', 'Angus', 'Charolais', 'Limousin'], value='Alentejana', description='Breed:', disabled=False)
p_Am_widget = widgets.FloatSlider(min=1000, max=8000, value=mammal._p_Am, description='$\{\dot{p}_{Am}\}$')
kap_X_widget = widgets.FloatSlider(min=0, max=1, value=mammal.kap_X, description='$\kappa_X$', step=0.05)
v_widget = widgets.FloatSlider(min=0.01, max=1, value=mammal._v, description='$\dot{v}$', step=0.01)
cow_selector.observe(default_values)

# display(p_Am_widget, kap_X_widget, cow_selector)
widgets.interactive(compute_metrics, breed=cow_selector, p_Am=p_Am_widget, kap_X=kap_X_widget, v=v_widget)

interactive(children=(Dropdown(description='Breed:', options=('Alentejana', 'Angus', 'Charolais', 'Limousin'),…

In [8]:
angus = pd.read_csv("../Data/Angus/angus_2020_processed.csv")
duration = 105
angus.head()

FileNotFoundError: [Errno 2] No such file or directory: '../Data/Angus/angus_2020_processed.csv'

In [ ]:
final_ages = [float(t) for t in angus['age'].values]
initial_ages = [t-105 for t in final_ages]
print(initial_ages, final_ages)

In [ ]:
mammal = pet.Ruminant(**pet.animals['bos_taurus_angus'])
def compare_deb_with_data(p_Am, v, p_M, kap_X, kappa):
    fig, axes = plt.subplots(1,2,figsize=(16,10))
    sns.scatterplot(data=angus, x='age', y='weight_gain', ax=axes[0])
    sns.scatterplot(data=angus, x='age', y='Inges MS', ax=axes[1])
    weight_gains_list = []
    feed_consumption_list = []
    format_strings = ['yo-', 'ro-']
    for i in range(2):
        mammal = pet.Ruminant(**pet.animals['bos_taurus_angus'])
        mammal._p_Am = p_Am[i]
        mammal._v = v[i]
        mammal._p_M = p_M[i]
        mammal.kap_X = kap_X[i]
        mammal.kappa = kappa[i]

        # Simulate Angus cow with DEB
        model = models.RUM(mammal)
        sol = model.simulate(food_function=1, t_span=(0, 450), step_size='auto', initial_state='birth')
        
        # Compute weight gains during experiment duration, assuming initial test for each data point
        weight_gains_deb = []
        for agei, agef in zip(initial_ages, final_ages):
            weight_gains_deb.append((sol[agef].wet_weight - sol[agei].wet_weight) / 1000)
        weight_gains_list.append(weight_gains_deb)
        axes[0].plot(final_ages, weight_gains_deb, format_strings[i])
        
        # Compute feed consumption
        feed_cons_deb = []
        for agei, agef in zip(initial_ages, final_ages):
            feed_cons_deb.append(sol.total_feed_intake(t1=sol.find_closest_time_step(agei),t2=sol.find_closest_time_step(agef))/1000)
        feed_consumption_list.append(feed_cons_deb)
        axes[1].plot(final_ages, feed_cons_deb, format_strings[i])
        
    # Weight plot
    axes[0].fill_between(final_ages, weight_gains_list[0], weight_gains_list[1], alpha=0.5, zorder=-1)
    axes[0].set_ylabel('Weight Gain during Test [kg]', fontsize=16)
    axes[0].set_xlabel('Age at end of Test [d]', fontsize=16)
    axes[0].grid()
    axes[0].set_xlim([340, 440])
    axes[0].set_ylim([0, 300])
    
    # Feed Consumption Plot
    axes[1].fill_between(final_ages, feed_consumption_list[0], feed_consumption_list[1], alpha=0.5, zorder=-1)
    axes[1].set_ylabel('Feed Consumption during Test [kg]', fontsize=16)
    axes[1].set_xlabel('Age at end of Test [d]', fontsize=16)
    axes[1].grid()
    axes[1].set_xlim([340, 440])
    axes[1].set_ylim([0, 1750])
    

# WIDGETS
p_Am_widget = widgets.FloatRangeSlider(min=1000, max=10000, value=(mammal._p_Am, mammal._p_Am), description='$\{\dot{p}_{Am}\}$', layout=widgets.Layout(width='40%'))
v_widget = widgets.FloatRangeSlider(min=0.01, max=0.25, value=(mammal._v, mammal._v), description='$\dot{v}$', step=0.001, layout=widgets.Layout(width='40%'))
p_M_widget = widgets.FloatRangeSlider(min=50, max=250, value=(mammal._p_M, mammal._p_M), description='$[\dot{p}_M]$', layout=widgets.Layout(width='40%'))
kap_X_widget = widgets.FloatRangeSlider(min=0, max=1, value=(mammal.kap_X, mammal.kap_X), description='$\kappa_X$', layout=widgets.Layout(width='40%'), step=0.01)
kappa_widget = widgets.FloatRangeSlider(min=0, max=1, value=(mammal.kappa, mammal.kappa), description='$\kappa$', layout=widgets.Layout(width='40%'), step=0.01)

widgets.interactive(compare_deb_with_data, p_Am=p_Am_widget, v=v_widget, p_M=p_M_widget, kap_X=kap_X_widget, kappa=kappa_widget)

In [ ]:
#mammal = pet.Ruminant(**pet.animals['bos_taurus_alentejana'])
mammal = pet.Pet(**pet.animals['sheep'])
feed_weight = pd.read_csv("../Data/ACBM/CSV Files/weight_intake_2021.csv")
feed_weight

In [ ]:

def cbrt(num):
    return np.power(num, 1/3)
f = 1
#mammal.kap_X = 0.999
alpha = mammal.p_Am * mammal.comp.X.w / mammal.comp.X.mu / mammal.kap_X * f
omega = 1 + f * mammal.E_m * mammal.comp.E.w / mammal.comp.E.d / mammal.comp.E.mu 
r_b = mammal.von_bertanlanffy_growth_rate(f)
Li = mammal.L_m*f
model = models.STD(mammal)
Lb = cbrt(model.get_state_at_maturity(mammal.E_Hb)[1])
Wi = cbrt(omega)*Li
Wb = cbrt(omega)*Lb

def tfi_vs_time(t):
    return alpha * (-0.5 / r_b * ((2*Li - (Li - Lb) * np.exp(-r_b*t))**2 - (Li + Lb)**2) + Li * Li * t)
def tfi_vs_length(L):
    return alpha / r_b * (-0.5  * ( (Li + L)**2 - (Li + Lb)**2 ) + Li * Li * np.log((Li-Lb)/(Li-L)))
def tfi_vs_weight(w):
    W = cbrt(w)
    return alpha * ((1/omega)**(2/3)) / r_b * (-0.5  * ( (Wi + W)**2 - (Wi + Wb)**2 ) + Wi * Wi * np.log((Wi-Wb)/(Wi-W)))

sol = model.simulate(food_function=f, t_span=(0, 4500), step_size='auto', initial_state='birth')

from scipy.integrate import cumulative_trapezoid

In [ ]:
print(tfi_vs_weight(10_000) / 1.034)
print(tfi_vs_weight(13_000) / 1.034)

In [ ]:
fig, ax = plt.subplots(figsize=(16,10))
t = np.linspace(0, 4500, 4500)
ax.plot(t, tfi_vs_time(t), label='Analytical', linewidth=3)

tfi = cumulative_trapezoid(-sol.feed_intake, sol.t, initial=0)
ax.plot(sol.t, tfi, '-.', label='Simulation', alpha=0.7, linewidth=3)

ax.grid()
ax.set_xlabel("Time [d]", fontsize=16)
ax.set_ylabel("Total Feed Intake [g]", fontsize=16)
ax.set_title("Total Feed Intake vs Time", fontsize=16)
ax.legend()
fig.savefig("Figures/tfi_time")

In [ ]:
fig, ax = plt.subplots(figsize=(16,10))
L = np.linspace(Lb, Li, len(sol.t))
ax.plot(L, tfi_vs_length(L), label='Analytical', linewidth=3)

simul_lengths = cbrt(sol.V)
print(simul_lengths)
tfi = cumulative_trapezoid(-sol.feed_intake, initial=0)

ax.plot(simul_lengths, tfi, '-.', label='Simulation', alpha=0.7, linewidth=3)
ax.grid()
ax.set_xlabel("Length [cm]", fontsize=16)
ax.set_ylabel("Total Feed Intake [g]", fontsize=16)
ax.set_title("Total Feed Intake vs Length", fontsize=16)
ax.legend()
fig.savefig("Figures/tfi_length")

In [ ]:
fig, ax = plt.subplots(figsize=(16,10))
W = np.linspace(Wb**3, Wi**3, len(sol.t))
ax.plot(W, tfi_vs_weight(W), label='Analytical', linewidth=3)


tfi = cumulative_trapezoid(-sol.feed_intake, initial=0)

ax.plot(sol.wet_weight, tfi, '-.', label='Simulation', alpha=0.7, linewidth=3)
ax.grid()
ax.set_xlabel("Wet Weight [g]", fontsize=16)
ax.set_ylabel("Total Feed Intake [g]", fontsize=16)
ax.set_title("Total Feed Intake vs Length", fontsize=16)
ax.legend()
fig.savefig("Figures/tfi_weight")

In [ ]:
mammal = pet.Ruminant(**pet.animals['ovis_aries_lacaune'])
mammal._p_Am = 32* 57.54/.955
model = models.RUM(mammal)
sol = model.simulate(food_function=0.7, t_span=(0, 33), step_size='auto', initial_state='birth')


In [ ]:
mammal._p_Am

In [ ]:
df = pd.read_csv("../Data/Sheep 4.0/lamb_weights_2022.csv")
df['dob'] = df['dob'].astype(str)
df.head()


In [ ]:
np.isnan(df.loc[0,'2022-02-18'])

In [ ]:
dates = ['2022-02-18', '2022-02-28', '2022-03-07']
weights_dict = {'id': [], 'age':[], 'weight': []}
for i in df.index.values:
    lamb_dob = dt.fromisoformat(df.loc[i,'dob'])
    lamb_id = df.loc[i,'id']
    weights_dict['id'].append(lamb_id)
    weights_dict['weight'].append(df.loc[i,'wb'])
    weights_dict['age'].append(0)
    for d in dates:
        d_dt = dt.fromisoformat(d)
        lamb_weight = df.loc[i,d]
        if lamb_weight == 0 or np.isnan(lamb_weight):
            continue
        weights_dict['id'].append(lamb_id)
        weights_dict['weight'].append(lamb_weight)
        weights_dict['age'].append((d_dt-lamb_dob).days)
lamb = pd.DataFrame(weights_dict)
lamb.head()

In [ ]:
avg_w = lamb.groupby(by='age').mean().weight
avg_w

In [ ]:
fig, ax = plt.subplots(figsize=(12,7.5))
ax.plot(sol.t, sol.wet_weight / 1000, label='Modelo DEB', linewidth=4)
ax.set_ylabel("Peso [kg]",fontsize=14)
ax.set_xlabel("Idade [d]",fontsize=14)

ax.plot(avg_w, 'o', label='Média dos Pesos')
ax.legend(fontsize=12)
ax.set_title("Evolução dos Pesos dos Borregos Nascidos em 2022", fontsize=14)